In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
sbert_embeddings.shape

(5568, 384)

## Baseline: BiLSTM

In [8]:
num_epochs = 100
hidden_dim_sizes = [200, 300, 384]
num_layers = 1
bidirectional = True
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_ids = torch.tensor(df_rumours["timeline_id"].astype(int))

In [9]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "hidden_dim_sizes": hidden_dim_sizes,
    "num_layers": num_layers,
    "bidirectional": bidirectional,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

## history_length=5

In [10]:
size = 5

In [11]:
bilstm_history_5, best_bilstm_history_5, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_5_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_5_focal_2_best_model.csv


In [12]:
bilstm_history_5.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1974987/1806291284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_5.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.658590  0.635418   0.633555   
                        0.0005         0.656850  0.632736   0.631099   
                        0.0010         0.654116  0.633482   0.632405   
           0.2          0.0001         0.659087  0.636001   0.634118   
                        0.0005         0.656664  0.632800   0.631099   
                        0.0010         0.653806  0.631944   0.630419   
300        0.1          0.0001         0.658590  0.635157   0.633296   
                        0.0005         0.656104  0.633609   0.631882   
                        0.0010         0.655918  0.635018   0.633391   
           0.2          0.0001         0.658403  0.635238   0.633463   
                        0.0005         0.656539  0.634503   0.632836   
                        0.0010         0.654738  0.634074   0.632681   
384        0.1          0.0001         0.656974  0.632455   0.630631   
                        0.0005         0.654675  0.631705   0.629987   
                        0.0010         0.659025  0.638396   0.636812   
           0.2          0.0001         0.657098  0.632166   0.630385   
                        0.0005         0.654365  0.632261   0.630877   
                        0.0010         0.661261  0.640628   0.639137   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.640687        0.775082  0.762978   
                        0.0005         0.637504        0.771892  0.759672   
                        0.0010         0.641020        0.761828  0.750673   
           0.2          0.0001         0.641316        0.775531  0.763445   
                        0.0005         0.637689        0.772611  0.760609   
                        0.0010         0.638280        0.765108  0.753294   
300        0.1          0.0001         0.640236        0.775307  0.763003   
                        0.0005         0.639391        0.769735  0.758009   
                        0.0010         0.641955        0.760435  0.748816   
           0.2          0.0001         0.640544        0.775981  0.763793   
                        0.0005         0.640628        0.770679  0.759225   
                        0.0010         0.641293        0.762232  0.750683   
384        0.1          0.0001         0.636861        0.776160  0.763668   
                        0.0005         0.637185        0.766500  0.754108   
                        0.0010         0.645615        0.761558  0.749748   
           0.2          0.0001         0.636342        0.776205  0.763397   
                        0.0005         0.638546        0.767534  0.755827   
                        0.0010         0.647992        0.762861  0.750689   

                                       valid_precision  valid_recall    k  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001                0.761679      0.764563  5.0   
                        0.0005                0.758421      0.761352  5.0   
                        0.0010                0.748874      0.754419  5.0   
           0.2          0.0001                0.762140      0.765017  5.0   
                        0.0005                0.759201      0.762521  5.0   
                        0.0010                0.751505      0.755847  5.0   
300        0.1          0.0001                0.761905      0.764360  5.0   
                        0.0005                0.756309      0.760424  5.0   
                        0.0010                0.746712      0.751778  5.0   
           0.2          0.0001                0.762735      0.765289  5.0   
                        0.0005                0.757337      0.761933  5.0   
                        0.0010                0.748736  

In [13]:
best_bilstm_history_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.663001,0.637499,"[0.7336476134354744, 0.5413495687468289]",0.635626,"[0.7545454545454545, 0.5167070217917675]",0.641168,"[0.7138761467889908, 0.5684603090037293]",None,0.776924,...,True,300,0.2,0.0001,1,focal,2,True,5,64
0,None,0.655359,0.634488,"[0.721829396720325, 0.5471467058535391]",0.632882,"[0.759417537195315, 0.5063463281958296]",0.641443,"[0.6877866972477065, 0.5950985615343634]",None,0.773285,...,True,300,0.2,0.0001,12,focal,2,True,5,64
0,None,0.656850,0.633727,"[0.7257559958289885, 0.541697784416231]",0.631880,"[0.7553488372093023, 0.508411214953271]",0.639021,"[0.698394495412844, 0.5796483750665956]",None,0.777733,...,True,300,0.2,0.0001,123,focal,2,True,5,64


In [14]:
best_bilstm_history_5["f1"].mean()

0.6352378441668978

In [15]:
best_bilstm_history_5["precision"].mean()

0.6334627323151567

In [16]:
best_bilstm_history_5["recall"].mean()

0.6405440975090383

In [17]:
np.stack(best_bilstm_history_5["f1_scores"]).mean(axis=0)

array([0.72707767, 0.54339802])

In [18]:
np.stack(best_bilstm_history_5["precision_scores"]).mean(axis=0)

array([0.75643728, 0.51048819])

In [19]:
np.stack(best_bilstm_history_5["recall_scores"]).mean(axis=0)

array([0.70001911, 0.58106908])

## history_length=11

In [20]:
size = 11

In [21]:
bilstm_history_11, best_bilstm_history_11, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_11_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_11_focal_2_best_model.csv


In [22]:
bilstm_history_11.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1974987/2881938888.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_11.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.678409  0.650060   0.648832   
                        0.0005         0.681578  0.657700   0.655792   
                        0.0010         0.685803  0.662397   0.660021   
           0.2          0.0001         0.677912  0.648955   0.647884   
                        0.0005         0.683131  0.659945   0.658025   
                        0.0010         0.685306  0.663310   0.660861   
300        0.1          0.0001         0.675986  0.648125   0.646733   
                        0.0005         0.676297  0.653972   0.651686   
                        0.0010         0.684995  0.660156   0.658166   
           0.2          0.0001         0.676856  0.649587   0.648106   
                        0.0005         0.678223  0.654675   0.652448   
                        0.0010         0.687729  0.661494   0.659916   
384        0.1          0.0001         0.680211  0.653979   0.652201   
                        0.0005         0.675179  0.651164   0.649146   
                        0.0010         0.683939  0.661289   0.658927   
           0.2          0.0001         0.679776  0.653993   0.652082   
                        0.0005         0.675427  0.647197   0.646239   
                        0.0010         0.685989  0.660229   0.658584   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.651706        0.868266  0.858939   
                        0.0005         0.662263        0.884396  0.877578   
                        0.0010         0.666866        0.881565  0.874725   
           0.2          0.0001         0.650298        0.868176  0.858590   
                        0.0005         0.664893        0.882374  0.875270   
                        0.0010         0.668903        0.878780  0.872002   
300        0.1          0.0001         0.650171        0.876533  0.868121   
                        0.0005         0.659514        0.880352  0.873577   
                        0.0010         0.663743        0.877881  0.870092   
           0.2          0.0001         0.651988        0.875006  0.866609   
                        0.0005         0.659314        0.882419  0.875536   
                        0.0010         0.664123        0.878151  0.870072   
384        0.1          0.0001         0.656783        0.871546  0.862678   
                        0.0005         0.655619        0.881925  0.875207   
                        0.0010         0.666376        0.871546  0.864039   
           0.2          0.0001         0.657063        0.871726  0.863080   
                        0.0005         0.649289        0.885025  0.877726   
                        0.0010         0.663195        0.870378  0.862057   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.863191      0.855471  11.0   
                        0.0005                0.877421      0.877918  11.0   
                        0.0010                0.874010      0.875496  11.0   
           0.2          0.0001                0.863698      0.854561  11.0   
                        0.0005                0.875657      0.875266  11.0   
                        0.0010                0.870718      0.873471  11.0   
300        0.1          0.0001                0.871334      0.865455  11.0   
                        0.0005                0.872592      0.874787  11.0   
                        0.0010                0.871551      0.868807  11.0   
           0.2          0.0001                0.869432      0.864288  11.0   
                        0.0005                0.875141      0.876027  11.0   
                        0.0010             

In [23]:
best_bilstm_history_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.677726,0.643878,"[0.7536686137626442, 0.5340878469415251]",0.644714,"[0.7490795808552818, 0.5403489640130862]",0.643142,"[0.7583142201834863, 0.5279701651571657]",None,0.880577,...,True,384,0.2,0.0005,1,focal,2,True,5,64
0,None,0.677353,0.649914,"[0.747924858016601, 0.5519026663215119]",0.648373,"[0.7599881621781592, 0.5367573011077543]",0.652083,"[0.7362385321100917, 0.5679275439531166]",None,0.885429,...,True,384,0.2,0.0005,12,focal,2,True,5,64
0,None,0.671202,0.647798,"[0.7385892116182573, 0.5570065293822201]",0.645629,"[0.7644171779141105, 0.5268408551068884]",0.652643,"[0.7144495412844036, 0.5908364411294619]",None,0.889069,...,True,384,0.2,0.0005,123,focal,2,True,5,64


In [24]:
best_bilstm_history_11["f1"].mean()

0.6471966210071266

In [25]:
best_bilstm_history_11["precision"].mean()

0.6462386735292134

In [26]:
best_bilstm_history_11["recall"].mean()

0.6492894073029544

In [27]:
np.stack(best_bilstm_history_11["f1_scores"]).mean(axis=0)

array([0.74672756, 0.54766568])

In [28]:
np.stack(best_bilstm_history_11["precision_scores"]).mean(axis=0)

array([0.75782831, 0.53464904])

In [29]:
np.stack(best_bilstm_history_11["recall_scores"]).mean(axis=0)

array([0.7363341 , 0.56224472])

## history_length=20

In [30]:
size = 20

In [31]:
bilstm_history_20, best_bilstm_history_20, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_20_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_20_focal_2_best_model.csv


In [32]:
bilstm_history_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1974987/1802967379.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.677105  0.650745   0.648948   
                        0.0005         0.671016  0.647986   0.645911   
                        0.0010         0.673377  0.652042   0.650006   
           0.2          0.0001         0.677477  0.651283   0.649517   
                        0.0005         0.671637  0.648241   0.646128   
                        0.0010         0.672134  0.652710   0.650888   
300        0.1          0.0001         0.664119  0.636858   0.635264   
                        0.0005         0.669835  0.647801   0.645646   
                        0.0010         0.667412  0.645366   0.643323   
           0.2          0.0001         0.664492  0.636366   0.634934   
                        0.0005         0.669338  0.645836   0.643718   
                        0.0010         0.671761  0.651067   0.649010   
384        0.1          0.0001         0.677788  0.649331   0.648105   
                        0.0005         0.672010  0.651207   0.649180   
                        0.0010         0.667971  0.645635   0.643529   
           0.2          0.0001         0.676918  0.649042   0.647964   
                        0.0005         0.670270  0.648079   0.645940   
                        0.0010         0.673501  0.650811   0.648597   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.653532        0.897426  0.890720   
                        0.0005         0.653197        0.906277  0.901047   
                        0.0010         0.658745        0.901829  0.896167   
           0.2          0.0001         0.654147        0.897021  0.890259   
                        0.0005         0.653183        0.907445  0.902202   
                        0.0010         0.660865        0.902098  0.896745   
300        0.1          0.0001         0.639527        0.901604  0.895242   
                        0.0005         0.653683        0.906771  0.901563   
                        0.0010         0.651287        0.902772  0.897040   
           0.2          0.0001         0.638583        0.900885  0.894554   
                        0.0005         0.650676        0.905693  0.900239   
                        0.0010         0.658035        0.902772  0.897280   
384        0.1          0.0001         0.650941        0.900616  0.894090   
                        0.0005         0.658104        0.904974  0.899703   
                        0.0010         0.651307        0.902682  0.897176   
           0.2          0.0001         0.650969        0.899358  0.892809   
                        0.0005         0.653895        0.904929  0.899520   
                        0.0010         0.656133        0.900571  0.894834   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.892928      0.888763  20.0   
                        0.0005                0.899652      0.902614  20.0   
                        0.0010                0.895501      0.897051  20.0   
           0.2          0.0001                0.892616      0.888233  20.0   
                        0.0005                0.901053      0.903444  20.0   
                        0.0010                0.895024      0.898785  20.0   
300        0.1          0.0001                0.897181      0.893520  20.0   
                        0.0005                0.900146      0.903126  20.0   
                        0.0010                0.896767      0.897383  20.0   
           0.2          0.0001                0.896143      0.893097  20.0   
                        0.0005                0.899500      0.901010  20.0   
                        0.0010             

In [33]:
best_bilstm_history_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.673998,0.652856,"[0.7385259381073405, 0.5671863400148479]",0.650604,"[0.7716338644173696, 0.5295748613678374]",0.659345,"[0.7081422018348624, 0.6105487480021311]",None,0.907265,...,True,200,0.2,0.0005,1,focal,2,True,5,64
0,None,0.666356,0.640259,"[0.7371512481644641, 0.5433673469387755]",0.638431,"[0.7555689343768814, 0.5212922173274597]",0.643502,"[0.7196100917431193, 0.567394778902504]",None,0.906996,...,True,200,0.2,0.0005,12,focal,2,True,5,64
0,None,0.674557,0.651606,"[0.741026401661228, 0.5621865596790371]",0.649349,"[0.7676705593116164, 0.5310279488394126]",0.656700,"[0.7161697247706422, 0.597229621736814]",None,0.908074,...,True,200,0.2,0.0005,123,focal,2,True,5,64


In [34]:
best_bilstm_history_20["f1"].mean()

0.6482406390942822

In [35]:
best_bilstm_history_20["precision"].mean()

0.6461280642734295

In [36]:
best_bilstm_history_20["recall"].mean()

0.6531825278316788

In [37]:
np.stack(best_bilstm_history_20["f1_scores"]).mean(axis=0)

array([0.7389012 , 0.55758008])

In [38]:
np.stack(best_bilstm_history_20["precision_scores"]).mean(axis=0)

array([0.76495779, 0.52729834])

In [39]:
np.stack(best_bilstm_history_20["recall_scores"]).mean(axis=0)

array([0.71464067, 0.59172438])

## history_length=35

In [40]:
size = 35

In [41]:
bilstm_history_35, best_bilstm_history_35, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_35_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_35_focal_2_best_model.csv


In [42]:
bilstm_history_35.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1974987/1307650554.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_35.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.682634  0.646920   0.649229   
                        0.0005         0.677167  0.646615   0.646126   
                        0.0010         0.674060  0.648593   0.646627   
           0.2          0.0001         0.682634  0.647981   0.649722   
                        0.0005         0.679341  0.651501   0.650230   
                        0.0010         0.676235  0.651370   0.649841   
300        0.1          0.0001         0.680398  0.645231   0.646941   
                        0.0005         0.676546  0.648278   0.647026   
                        0.0010         0.676794  0.652131   0.650176   
           0.2          0.0001         0.680398  0.644844   0.646736   
                        0.0005         0.678472  0.648248   0.647678   
                        0.0010         0.664057  0.642958   0.641035   
384        0.1          0.0001         0.678596  0.647373   0.647139   
                        0.0005         0.680273  0.650986   0.650090   
                        0.0010         0.673874  0.650197   0.648008   
           0.2          0.0001         0.679031  0.647518   0.647414   
                        0.0005         0.673812  0.645943   0.644646   
                        0.0010         0.673812  0.650420   0.648252   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.645523        0.904165  0.897754   
                        0.0005         0.647264        0.914499  0.909192   
                        0.0010         0.652052        0.909916  0.904812   
           0.2          0.0001         0.646835        0.903446  0.897043   
                        0.0005         0.653489        0.914948  0.910045   
                        0.0010         0.655406        0.912073  0.907025   
300        0.1          0.0001         0.643967        0.902907  0.896721   
                        0.0005         0.650026        0.914723  0.909631   
                        0.0010         0.656205        0.908658  0.903348   
           0.2          0.0001         0.643433        0.902548  0.896245   
                        0.0005         0.649047        0.914184  0.908857   
                        0.0010         0.649650        0.909287  0.904521   
384        0.1          0.0001         0.647666        0.905288  0.899541   
                        0.0005         0.652155        0.915397  0.910083   
                        0.0010         0.654821        0.904435  0.899330   
           0.2          0.0001         0.647673        0.905243  0.899436   
                        0.0005         0.648129        0.912567  0.907434   
                        0.0010         0.655265        0.907939  0.902834   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.900700      0.895227  35.0   
                        0.0005                0.910104      0.908352  35.0   
                        0.0010                0.903654      0.906075  35.0   
           0.2          0.0001                0.899737      0.894737  35.0   
                        0.0005                0.909271      0.910888  35.0   
                        0.0010                0.906234      0.907976  35.0   
300        0.1          0.0001                0.898282      0.895297  35.0   
                        0.0005                0.909590      0.909688  35.0   
                        0.0010                0.902713      0.904017  35.0   
           0.2          0.0001                0.898197      0.894486  35.0   
                        0.0005                0.909809      0.908053  35.0   
                        0.0010             

In [43]:
best_bilstm_history_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.676608,0.644514,"[0.7513257847212269, 0.537703170796696]",0.644532,"[0.7512181140728003, 0.5378464818763327]",0.644497,"[0.7514334862385321, 0.537559936068194]",None,0.915218,...,True,384,0.1,0.0005,1,focal,2,True,5,64
0,None,0.675862,0.647330,"[0.747641851690611, 0.5470174524615785]",0.646074,"[0.7569791360564208, 0.5351681957186545]",0.648968,"[0.7385321100917431, 0.5594033031433138]",None,0.917509,...,True,384,0.1,0.0005,12,focal,2,True,5,64
0,None,0.688350,0.661112,"[0.7571884984025558, 0.5650364203954213]",0.659663,"[0.7672160094173043, 0.5521098118962887]",0.663001,"[0.7474197247706422, 0.5785828449653703]",None,0.913465,...,True,384,0.1,0.0005,123,focal,2,True,5,64


In [44]:
best_bilstm_history_35["f1"].mean()

0.6509855297446816

In [45]:
best_bilstm_history_35["precision"].mean()

0.6500896248396336

In [46]:
best_bilstm_history_35["recall"].mean()

0.6521552342129658

In [47]:
np.stack(best_bilstm_history_35["f1_scores"]).mean(axis=0)

array([0.75205204, 0.54991901])

In [48]:
np.stack(best_bilstm_history_35["precision_scores"]).mean(axis=0)

array([0.75847109, 0.54170816])

In [49]:
np.stack(best_bilstm_history_35["recall_scores"]).mean(axis=0)

array([0.74579511, 0.55851536])

## history_length=80

In [50]:
size = 80

In [51]:
bilstm_history_80, best_bilstm_history_80, _, __ = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_80_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_80_focal_2_best_model.csv


In [52]:
bilstm_history_80.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1974987/773949101.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_80.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.689158  0.653583   0.656255   
                        0.0005         0.690587  0.660368   0.660381   
                        0.0010         0.688599  0.659309   0.658625   
           0.2          0.0001         0.685182  0.650908   0.652495   
                        0.0005         0.692637  0.662399   0.662389   
                        0.0010         0.677975  0.648109   0.647357   
300        0.1          0.0001         0.686362  0.653336   0.654308   
                        0.0005         0.692327  0.663965   0.663061   
                        0.0010         0.682448  0.654032   0.653293   
           0.2          0.0001         0.686673  0.653942   0.654760   
                        0.0005         0.693135  0.663724   0.663304   
                        0.0010         0.684933  0.652061   0.653011   
384        0.1          0.0001         0.685120  0.652245   0.653034   
                        0.0005         0.687294  0.661807   0.659857   
                        0.0010         0.685368  0.656089   0.655408   
           0.2          0.0001         0.682821  0.650641   0.651015   
                        0.0005         0.693445  0.665603   0.664512   
                        0.0010         0.686921  0.658232   0.657321   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.652057        0.899897  0.893313   
                        0.0005         0.660702        0.913106  0.907924   
                        0.0010         0.660076        0.910096  0.905038   
           0.2          0.0001         0.649778        0.902817  0.896432   
                        0.0005         0.662607        0.914499  0.909312   
                        0.0010         0.649116        0.908568  0.903501   
300        0.1          0.0001         0.652532        0.905738  0.899798   
                        0.0005         0.665116        0.910545  0.905459   
                        0.0010         0.655960        0.908254  0.903072   
           0.2          0.0001         0.653222        0.905917  0.900041   
                        0.0005         0.664302        0.910949  0.905748   
                        0.0010         0.651392        0.907535  0.902001   
384        0.1          0.0001         0.651576        0.907984  0.902291   
                        0.0005         0.664814        0.909916  0.904778   
                        0.0010         0.657058        0.907625  0.902131   
           0.2          0.0001         0.650300        0.906816  0.901056   
                        0.0005         0.667002        0.908838  0.903315   
                        0.0010         0.659401        0.907220  0.901708   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.895749      0.891235  80.0   
                        0.0005                0.907854      0.908019  80.0   
                        0.0010                0.903746      0.906469  80.0   
           0.2          0.0001                0.898806      0.894364  80.0   
                        0.0005                0.909675      0.909008  80.0   
                        0.0010                0.901931      0.905235  80.0   
300        0.1          0.0001                0.901053      0.898715  80.0   
                        0.0005                0.904368      0.906628  80.0   
                        0.0010                0.902001      0.904392  80.0   
           0.2          0.0001                0.901098      0.899110  80.0   
                        0.0005                0.905232      0.906321  80.0   
                        0.0010             

In [53]:
best_bilstm_history_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.688910,0.655178,"[0.7630271191253727, 0.5473284513154325]",0.656648,"[0.7558368495077356, 0.5574585635359116]",0.653958,"[0.770355504587156, 0.537559936068194]",None,0.915083,...,True,200,0.2,0.0005,1,focal,2,True,5,64
0,None,0.689655,0.662103,"[0.7585906916050458, 0.5656144012522828]",0.660786,"[0.7673804634790261, 0.5541922290388548]",0.663759,"[0.75, 0.5775173148641449]",None,0.914409,...,True,200,0.2,0.0005,12,focal,2,True,5,64
0,None,0.699348,0.669916,"[0.7684799770345916, 0.5713526441668881]",0.669732,"[0.7694739867778098, 0.5699893955461294]",0.670105,"[0.7674885321100917, 0.5727224294086308]",None,0.914005,...,True,200,0.2,0.0005,123,focal,2,True,5,64


In [54]:
best_bilstm_history_80["f1"].mean()

0.6623988807499356

In [55]:
best_bilstm_history_80["precision"].mean()

0.6623885813142446

In [56]:
best_bilstm_history_80["recall"].mean()

0.6626072861730362

In [57]:
np.stack(best_bilstm_history_80["f1_scores"]).mean(axis=0)

array([0.76336593, 0.56143183])

In [58]:
np.stack(best_bilstm_history_80["precision_scores"]).mean(axis=0)

array([0.76423043, 0.56054673])

In [59]:
np.stack(best_bilstm_history_80["recall_scores"]).mean(axis=0)

array([0.76261468, 0.56259989])